In [ ]:
#hide 
#Standard Lib imports
import sys, os
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

In [ ]:
#hide
%load_ext autoreload
%autoreload 2 
%config Completer.use_jedi = False

In [ ]:
from rfpye.parser import parse_bin, export_metadata, extract_level
from rfpye.utils import get_files, df_optimize
import numpy as np
import pandas as pd
from fastcore.foundation import L
from rich import print

In [ ]:
entrada = Path(r'D:\OneDrive - ANATEL\Remote')
saida = Path(r'C:\Users\rsilva\Downloads\saida')

In [ ]:
arquivos = get_files(entrada, extensions=['.bin']) ;  arquivos

(#65) [Path('D:/OneDrive - ANATEL/Remote/rfeye002073/rfeye002073_210620_T231206.bin'),Path('D:/OneDrive - ANATEL/Remote/rfeye002080/rfeye002080_691231_T210111.bin'),Path('D:/OneDrive - ANATEL/Remote/rfeye002081/rfeye002081_210620_T232204.bin'),Path('D:/OneDrive - ANATEL/Remote/rfeye002083/rfeye002083_210621_T160001.bin'),Path('D:/OneDrive - ANATEL/Remote/rfeye002084/rfeye002084_210623_T144012.bin'),Path('D:/OneDrive - ANATEL/Remote/rfeye002086/rfeye002086_210622_T004723.bin'),Path('D:/OneDrive - ANATEL/Remote/rfeye002087/rfeye002087_210628_T224301.bin'),Path('D:/OneDrive - ANATEL/Remote/RFeye002090-VCP/rfeye002090-VCP_210623_T024236.bin'),Path('D:/OneDrive - ANATEL/Remote/rfeye002091/rfeye002091_210618_T145747.bin'),Path('D:/OneDrive - ANATEL/Remote/rfeye002092/rfeye002092_210603_T205009.bin')...]

In [ ]:
arquivo = arquivos[-1]

In [ ]:
with open(arquivo, mode="rb") as bfile:
    # O primeiro bloco do arquivo é o cabeçalho e tem 36 bytes de tamanho.
    header = bfile.read(36)
    body = bfile.read()

In [ ]:
body = body.split(b'UUUU')

In [ ]:
body = L(body)

In [ ]:
np.frombuffer(body[3][-4:], dtype=np.uint32).item()

2199

In [ ]:
np.frombuffer(body[3][:-4], dtype=np.uint8).sum().astype(np.uint32).item()

2199

In [ ]:
body.map(lambda x: len(x[12:-4]))

(#44261) [64,68,120,40,3732,1148,40,3732,1148,40...]

In [ ]:
%%time
btypes = [21, 40, 67, 68]
parsed_bin = parse_bin(arquivo, btypes)

Wall time: 3.27 s


In [ ]:
blocks = parsed_bin['blocks']

In [ ]:
blocks

defaultdict(fastcore.foundation.L,
            {(21,
              0): (#1) [<rfpye.blocks.DType21 object at 0x000001F630AF88C8>],
             (40,
              1): (#9057) [<rfpye.blocks.DType40 object at 0x000001F630AF8948>,<rfpye.blocks.DType40 object at 0x000001F630AF8F08>,<rfpye.blocks.DType40 object at 0x000001F635B39488>,<rfpye.blocks.DType40 object at 0x000001F635B399C8>,<rfpye.blocks.DType40 object at 0x000001F635B46D08>,<rfpye.blocks.DType40 object at 0x000001F635B476C8>,<rfpye.blocks.DType40 object at 0x000001F635B47F48>,<rfpye.blocks.DType40 object at 0x000001F635B49808>,<rfpye.blocks.DType40 object at 0x000001F630AE2688>,<rfpye.blocks.DType40 object at 0x000001F635B47688>...],
             (67,
              300): (#9057) [<rfpye.blocks.DType67 object at 0x000001F630AF8A48>,<rfpye.blocks.DType67 object at 0x000001F630AF8FC8>,<rfpye.blocks.DType67 object at 0x000001F635B39548>,<rfpye.blocks.DType67 object at 0x000001F635B3C308>,<rfpye.blocks.DType67 object at 0x000001F635

In [ ]:
d = {}
for (btype, tid), list_blocks in blocks.items():
    if btype == 67:
        d[btype, tid] = list_blocks.map(lambda x: len(x.data[x.start:x.stop]) != x.ndata).sum()

In [ ]:
d

{(67, 300): 1,
 (67, 310): 0,
 (67, 100): 0,
 (67, 110): 0,
 (67, 120): 0,
 (67, 130): 0,
 (67, 200): 73,
 (67, 210): 165,
 (67, 220): 3,
 (67, 230): 78,
 (67, 240): 100,
 (67, 250): 231,
 (67, 320): 0,
 (67, 330): 0,
 (67, 340): 115,
 (67, 350): 55,
 (67, 360): 93,
 (67, 370): 129,
 (67, 380): 6,
 (67, 390): 10}

In [ ]:
%%time
metadata = export_metadata(blocks)

In [ ]:
metadata.keys()

In [ ]:
metadata[40, 1].head()

In [ ]:
metadata[67, 300].iloc[0]

In [ ]:
filter_attrs = {21: ['hostname', 'method'],
                40: ['altitude', 'latitude', 'longitude'],
                67: ['wallclock_datetime', 'sample', 'minimum', 'ndata', 'processing', 'bw', 'data_type', 'description', 'start_mega', 'stop_mega']}
metadata = export_metadata(blocks, filter_attrs)

In [ ]:
metadata[21,0]

,hostname,method
0,rfeye002345,ScriptRFeye2021_v2.cfg


In [ ]:
metadata[40, 1]

,altitude,latitude,longitude
0,19.799999,-29.990238,-51.169189
1,21.000000,-29.990234,-51.169201
2,20.200001,-29.990240,-51.169201
3,17.900000,-29.990227,-51.169209
4,14.500000,-29.990217,-51.169209
...,...,...,...
9052,15.100000,-29.990221,-51.169212
9053,13.000000,-29.990211,-51.169220
9054,11.800000,-29.990223,-51.169205
9055,19.299999,-29.990215,-51.169186


In [ ]:
metadata[67, 120].head()

,sample,minimum,ndata,processing,bw,data_type,description,start_mega,stop_mega
wallclock_datetime,,,,,,,,,
2021-06-24 11:15:00.931540,5858,-74.5,1280,peak,73828,dBμV/m,PRD 2021 (Faixa principal 3 de 4).,170,220
2021-06-24 11:20:00.833120,5858,-76.5,1280,peak,73828,dBμV/m,PRD 2021 (Faixa principal 3 de 4).,170,220
2021-06-24 11:25:00.832160,5860,-74.5,1280,peak,73828,dBμV/m,PRD 2021 (Faixa principal 3 de 4).,170,220
2021-06-24 11:30:00.831240,5855,-72.5,1280,peak,73828,dBμV/m,PRD 2021 (Faixa principal 3 de 4).,170,220
2021-06-24 11:35:00.832530,5852,-74.5,1280,peak,73828,dBμV/m,PRD 2021 (Faixa principal 3 de 4).,170,220


In [ ]:
out = dict()
out['File_Name'] = arquivo.name
out['File_Version'] = parsed_bin['file_version']
out['File_Type'] = 'RFEye Logger Trace'
out['Device'] = 'Rfeye Node' 
out['Fluxos'] = dict()
fluxos = L()
for (btype, tid), df in metadata.items():
    if btype == 21:
        out['Equipment_ID'] = df.hostname.item()
        out['Script_Version'] = df.method.item()
    elif btype == 40:
        out['Latitude'] = df.latitude.mean()
        out['Longitude'] = df.longitude.mean()
        out['Altitude'] = df.altitude.mean()
        out['Count_GPS'] = df.shape[0]
        out['Sum_Latitude']  = df.latitude.sum()
        out['Sum_Longitude'] = df.longitude.sum()
    elif btype == 67:
        timestamp = df.index.values        
        level = dict()
        level['Initial_Time'] = timestamp.min()
        level['Final_Time'] = timestamp.max()
        level['Sample_Duration'] = df['sample'].median()        
        fluxo = df.drop(['minimum', 'sample'], axis=1).iloc[0]
        level['Description'] = fluxo.description
        level['Start_Frequency'] = fluxo.start_mega
        level['Stop_Frequency'] = fluxo.stop_mega
        level['Trace_Type'] = fluxo.processing
        level['RBW'] = fluxo.bw
        level['Level_Units'] = fluxo.data_type
        level['Vector_Length'] = fluxo.ndata
        level['Timestamp'] = timestamp
        level_matrix = extract_level(blocks[(btype, tid)], dtype=np.float16)
        level['Level_Data'] = level_matrix
        out['Fluxos'][(btype, tid)] =  level

In [ ]:
print({k:v for k,v in out.items() if k!= 'Fluxos'})

{
    'File_Name': 'rfeye002345_210624_T111158.bin',
    'File_Version': 23,
    'File_Type': 'RFEye Logger Trace',
    'Device': 'Rfeye Node',
    'Equipment_ID': 'rfeye002345',
    'Script_Version': 'ScriptRFeye2021_v2.cfg',
    'Latitude': -29.990221,
    'Longitude': -51.169205,
    'Altitude': 18.73128,
    'Count_GPS': 9057,
    'Sum_Latitude': -271621.44,
    'Sum_Longitude': -463439.5
}

In [ ]:
print(out['Fluxos'].keys())

dict_keys([(67, 300), (67, 310), (67, 100), (67, 110), (67, 120), (67, 130), (67, 200), (67, 
210), (67, 220), (67, 230), (67, 240), (67, 250), (67, 320), (67, 330), (67, 340), (67, 350),
(67, 360), (67, 370), (67, 380), (67, 390)])

In [ ]:
print(out['Fluxos'][67,120])

{
    'Initial_Time': numpy.datetime64('2021-06-24T11:15:00.931540000'),
    'Final_Time': numpy.datetime64('2021-06-30T18:05:00.850530000'),
    'Sample_Duration': 5856.0,
    'Description': 'PRD 2021 (Faixa principal 3 de 4).',
    'Start_Frequency': 170,
    'Stop_Frequency': 220,
    'Trace_Type': 'peak',
    'RBW': 73828,
    'Level_Units': 'dBμV/m',
    'Vector_Length': 1280,
    'Timestamp': array(['2021-06-24T11:15:00.931540000', '2021-06-24T11:20:00.833120000',
       '2021-06-24T11:25:00.832160000', ...,
       '2021-06-30T17:55:00.849980000', '2021-06-30T18:00:00.851040000',
       '2021-06-30T18:05:00.850530000'], dtype='datetime64[ns]'),
    'levels': array([[42. , 41. , 41. , ..., 44. , 41.5, 45. ],
       [38.5, 41. , 39.5, ..., 40. , 40. , 41.5],
       [46. , 43. , 40. , ..., 37.5, 37.5, 46.5],
       ...,
       [35.5, 38.5, 39.5, ..., 33.5, 35.5, 37. ],
       [33. , 32. , 32. , ..., 38. , 42. , 47.5],
       [44. , 41. , 32.5, ..., 46. , 45. , 42.5]], dtype=float16)
}

In [ ]:
levels_f16 = out['Fluxos'][67,120]['Level_Data']

In [ ]:
pd.to_pickle(out, saida / 'teste_float_16.pkl')

In [ ]:
out = dict()
out['File_Name'] = arquivo.name
out['File_Version'] = parsed_bin['file_version']
out['File_Type'] = 'RFEye Logger Trace'
out['Device'] = 'Rfeye Node' 
out['Fluxos'] = dict()
fluxos = L()
for (btype, tid), df in metadata.items():
    if btype == 21:
        out['Equipment_ID'] = df.hostname.item()
        out['Script_Version'] = df.method.item()
    elif btype == 40:
        out['Latitude'] = df.latitude.median()
        out['Longitude'] = df.longitude.median()
        out['Altitude'] = df.altitude.median()
        out['Count_GPS'] = df.shape[0]
    elif btype == 67:
        timestamp = df.index.values        
        level = dict()
        level['Initial_Time'] = timestamp.min()
        level['Sample_Duration'] = df['sample'].median()        
        fluxo = df.drop(['minimum', 'sample'], axis=1).iloc[0]
        level['Description'] = fluxo.description
        level['Start_Frequency'] = fluxo.start_mega
        level['Stop_Frequency'] = fluxo.stop_mega
        level['Trace_Type'] = fluxo.processing
        level['RBW'] = fluxo.bw
        level['Level_Units'] = fluxo.data_type
        level['Vector_Length'] = fluxo.ndata
        level['Timestamp'] = timestamp
        level['Minimum'] = df.minimum.values.astype('float16')
        level['Level_Data'] = extract_level(blocks[(btype, tid)], dtype=np.uint8)
#        level_matrix.index = timestamp
#        level['Levels'] = level_matrix
        out['Fluxos'][(btype, tid)] =  level

In [ ]:
print(out['Fluxos'][67,120])

{
    'Initial_Time': numpy.datetime64('2021-06-24T11:15:00.931540000'),
    'Sample_Duration': 5856.0,
    'Description': 'PRD 2021 (Faixa principal 3 de 4).',
    'Start_Frequency': 170,
    'Stop_Frequency': 220,
    'Trace_Type': 'peak',
    'RBW': 73828,
    'Level_Units': 'dBμV/m',
    'Vector_Length': 1280,
    'timestamp': array(['2021-06-24T11:15:00.931540000', '2021-06-24T11:20:00.833120000',
       '2021-06-24T11:25:00.832160000', ...,
       '2021-06-30T17:55:00.849980000', '2021-06-30T18:00:00.851040000',
       '2021-06-30T18:05:00.850530000'], dtype='datetime64[ns]'),
    'Minimum': array([-74.5, -76.5, -74.5, ..., -76.5, -76.5, -75.5], dtype=float16),
    'Level_Data': array([[233, 231, 231, ..., 237, 232, 239],
       [230, 235, 232, ..., 233, 233, 236],
       [241, 235, 229, ..., 224, 224, 242],
       ...,
       [224, 230, 232, ..., 220, 224, 227],
       [219, 217, 217, ..., 229, 237, 248],
       [239, 233, 216, ..., 243, 241, 236]], dtype=uint8)
}

In [ ]:
pd.to_pickle(out, saida / 'teste_meta_int8.pkl')

In [ ]:
levels = out['Fluxos'][67,120]['levels']
minimum = out['Fluxos'][67,120]['minimum']
timestamp = out['Fluxos'][67,120]['timestamp']
frequencies = out['Fluxos'][67,120]['frequencies']

In [ ]:
levels = (levels / 2 + minimum.reshape(-1,1)).astype(np.float16)

In [ ]:
(levels_f16 == levels).all()

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
import json

In [ ]:
meta_key = arquivo.name

In [ ]:
table = pa.Table.from_pandas(out['Fluxos'][67,120]['Levels'])

In [ ]:
print(table.schema.metadata)

In [ ]:
custom_meta_json = json.dumps({k:str(v) for k,v in out.items() if k!= 'Fluxos'})

In [ ]:
existing_meta = table.schema.metadata

combined_meta = {
    meta_key.encode() : custom_meta_json.encode(),
    **existing_meta
}

In [ ]:
table = table.replace_schema_metadata(combined_meta)

In [ ]:
print(table.schema.metadata)

In [ ]:
pq.write_table(table, f'{saida}/example.parquet', compression='GZIP')

In [ ]:
restored_table = pq.read_table(f'{saida}/example.parquet')

In [ ]:
restored_df = restored_table.to_pandas()

In [ ]:
restored_df.head()

In [ ]:
restored_meta_json = restored_table.schema.metadata[meta_key.encode()]

In [ ]:
restored_table.schema.metadata.keys()

In [ ]:
restored_meta_json

In [ ]:
dict.update()

In [ ]:
header_key = arquivo.name
header_meta_json = json.dumps({k:str(v) for k,v in out.items() if k!= 'Fluxos'})
combined_meta = {header_key.encode(): header_meta_json.encode()}

for (btype, tid), fluxo in out['Fluxos'].items():
    combined_meta = {header_key.encode(): header_meta_json.encode()}
    meta_key = f'{btype}-{tid}_meta'
    table = pa.Table.from_pandas(fluxo['Levels'], nthreads=os.cpu_count())
    existing_meta = table.schema.metadata
    custom_meta = {k:str(v) for k,v in fluxo.items() if k not in ('Timestamp', 'Minimum')}
    custom_meta['Minimum'] = fluxo['Minimum'].tolist()
    custom_meta_json = json.dumps(custom_meta)
    existing_meta.update({meta_key.encode() : custom_meta_json.encode()})
    existing_meta[f'{btype}-{tid}_levels'.encode()] = existing_meta.pop(b'pandas')
    combined_meta.update(existing_meta)
    table = table.replace_schema_metadata(combined_meta)
    print(table.schema.metadata.keys())
    pq.write_table(table, f'{saida}/{header_key}_{btype}-{tid}.parquet', compression='GZIP')   

dict_keys([b'rfeye002345_210624_T111158.bin', b'67-300_meta', b'67-300_levels'])

dict_keys([b'rfeye002345_210624_T111158.bin', b'67-310_meta', b'67-310_levels'])

dict_keys([b'rfeye002345_210624_T111158.bin', b'67-100_meta', b'67-100_levels'])

dict_keys([b'rfeye002345_210624_T111158.bin', b'67-110_meta', b'67-110_levels'])

dict_keys([b'rfeye002345_210624_T111158.bin', b'67-120_meta', b'67-120_levels'])

...

dict_keys([b'rfeye002345_210624_T111158.bin', b'67-350_meta', b'67-350_levels'])

dict_keys([b'rfeye002345_210624_T111158.bin', b'67-360_meta', b'67-360_levels'])

dict_keys([b'rfeye002345_210624_T111158.bin', b'67-370_meta', b'67-370_levels'])

dict_keys([b'rfeye002345_210624_T111158.bin', b'67-380_meta', b'67-380_levels'])

dict_keys([b'rfeye002345_210624_T111158.bin', b'67-390_meta', b'67-390_levels'])

In [ ]:
restored_table = pq.read_table(f'{saida}/{header_key}_67-300.parquet')

In [ ]:
restored_df = restored_table.to_pandas()

In [ ]:
from math import pi
from bokeh.io import show, output_notebook
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, PrintfTickFormatter
from bokeh.plotting import figure

In [ ]:
data = pd.DataFrame(levels, index=timestamp, columns=frequencies)
data.columns.name = "Frequencies"
data.index.name = "Time"
data.head()

In [ ]:
df = pd.DataFrame(data.stack(), columns=['Levels']).reset_index() ; df.head()

In [ ]:
frequencies

NameError: name 'frequencies' is not defined

In [ ]:
time = timestamp.tolist()
frequencies = frequencies.astype('float32').tolist()

In [ ]:
# this is the colormap from the original NYTimes plot
colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette=colors, low=df.Levels.min(), high=df.Levels.max())

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title="Waterfall ({0} - {1})".format(timestamp[0], timestamp[-1]),
           x_range=frequencies, y_range=timestamp,
           x_axis_label = 'Frequencies',
           y_axis_label = 'Timestamp',
           x_axis_location="above", plot_width=1920, plot_height=1080,
           tools=TOOLS, toolbar_location='below',
           tooltips=[('Frequência', '@Frequencies'), ('Tempo', '@Time'), ('Nível', '@Levels')])

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "0px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = pi / 3

p.rect(x="Frequencies", 
       y="Time", 
       width=1,
       height=1, 
       source=df,
       fill_color={'field': 'Levels', 'transform': mapper}, 
       line_color=None)



color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="10px",
                 ticker=BasicTicker(desired_num_ticks=len(colors)),
                 formatter=PrintfTickFormatter(format="%ddBm"),
                 label_standoff=6, border_line_color=None)

p.add_layout(color_bar, 'right')

show(p)      # show the plot